# Inverse Distance and Local Statistics 

Created by: Derek Robinson <br>
Last Updated: February 4, 2021

### Learning Objectives

In this Jupyter Notebook you will learn or review 
<ul>
    <li> about inverse power relationships, </li>
    <li> pair-wise two-way relationships using matrices,</li>
    <li> create areal unit count data, </li>
    <li> conduct hot-spot analysis using the Local Gi statistic, </li>
    <li> create different representations of neighbourhoods and spatial weights. </li>
</ul>

### Assignment Completion

For those in GEOG 318/PLAN 353 @UWaterloo, the assignment directions and submission requirements are provided on LEARN as a quiz that spans multiple days. To complete the quiz/assignment you will need to do the following:

<ul>
    <li> Complete this Jupyter Notebook.</li>
    <ul>
        <li> To advance to the next text cell or execute the code in a cell hold <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift</mark></font> and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Enter</mark></font> </li>
        <li> If there is a method for which you would like to know more about its parameters or how it works then you can place your cursor on the method and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift + Tab</mark></font> and a help box will appear that you can expand and scroll through.</li>
    </ul>
    <li> Complete the Inverse Distance and Local Statistics Quiz/Assignment in LEARN, which may require you to </li>
    <ul>
        <li> copy results from your notebook into the LEARN input boxes, </li>
        <li> take screen captures of your notebook output and submit them via LEARN, or </li>
        <li> change values in the Jupyter Notebook and re-execute cells/code to obtain new results. </li>
    </ul>
    <li> <b>While you may discuss course content with your classmates, you are to complete the assignment individually.</b></li>
</ul>

### Problem Statement & Data

The local statistics calculated below use point pattern data of Home Depot stores located in Ontario, Canada, in 2014. These data along with Census data and other store and spatial data were used to conduct analyses for a major home improvement company. The following publication is one output of our retail analysis that can provide some context and illustrate additional types of spatial analysis that can be conducted.

Robinson, D.T., and A. Balulescu, 2018. Comparison of Methods for Quantifying Consumer Spending on Retail using Publicly Available Data. International Journal of Geographic Information Science, 32(6), 1061-1086.

**While the methods below use store location data, they are applicable to point data representing other phenomena (e.g., wetland, wildlife, school, or crime locations).** 

### Lets get started
Sometimes when running functions and code in R you will receive warnings. These are displayed in pink in Jupyter Notebook following the execution of a cell. The warnings arise for a variety of reasons like a parameter was not specified so a default will be used. In creating these notebooks I have reviewed the warnings and they do not affect the results. The next line of code turns them off so that you are not confused or worried about them. However, if you would like to see them you may restart the notebook and comment the following cell code or skip executing it.

In [ ]:
options(warn=-1)
# command to turn warnings on
# options(warn=0)

We need to use several methods in the `rgdal`, `sp`, `spatstat`, and `sf` packages. In addition to the spatial analysis methods from those packages we will use `ggplot2`, `ggmap`, `ggspatial`, and `patchwork` packages for visualizing the data. First try loading the packages into your r library and if you get an error then uncomment the install command and execute the cell to install the package.

In [ ]:
#install.packages("rgdal")
#install.packages("sp")
#install.packages("spatstat")
#install.packages("sf")
library(rgdal)
library(sp)
library(spatstat)
library(sf)

#install.packages("ggplot2")
#install.packages("ggmap")
#install.packages("ggspatial")
#install.packages("patchwork")
library(ggplot2)
library(ggmap)
library(ggspatial)
library(patchwork)

Import the shapefiles we will work with using the `readOGR()` function from `rgdal`. This function allows us to define a shapefile as a usable object in R. The first input is the data source name (dsn) of the file (e.g., `CD_ON_2011.shp`. The second input requirement is `layer`, which is the layer name (e.g., `CD_ON_2011`. Note that the specifications for dsn and layer may vary with the rgdal function being used, but for now we can use them as text descriptors of our file. 

<b>It may take some time to read in the Census data because they have many features and many attributes (especially if you are working on your home machine).</b>

In [ ]:
CD <- readOGR("CD_ON_2011.shp", "CD_ON_2011")
#CSD <- readOGR("CSD_ON_2011.shp", "CSD_ON_2011")
#DA <- readOGR("DA_ON_2011.shp", "DA_ON_2011")
OHD <- readOGR("OntarioHomeDepot.shp", "OntarioHomeDepot")
OP <- readOGR("OntarioProvince.shp", "OntarioProvince")

Lets map the data to see the distribution of Census Divisions within the Province of Ontario. To do this we will use `ggplot2`, which uses a data.frame for plotting. We can coerce the data into a data.frame using the `data.frame()` function, however to show you another approach we will use the `fortify()` function.

You can try plotting the different shapefiles if you like, but be careful as the census geographies can take a long time to plot and you should not plot them all simultaneously. If you are working on the JupyterHub then plotting will be much faster. If you would like to take a quick look at the data in the shapefiles then you can do so using `data.frame(CD)`. There are other ways to visualize the spatial and tabular data in a shapefile. Feel free to explore these methods and provide suggestions for modifications to this assignment.

In [ ]:
CD_data <- fortify(CD)
#CSD_data <- fortify(CSD)
#DA_data <- fortify(DA)
OHD_data <- data.frame(OHD)
OP_data <- fortify(OP)

g <- ggplot() + geom_polygon(data=CD_data, aes(x=long, y=lat, group=group), color="black", fill="green") + xlab("Longitude") + ylab("Latitude") + ggtitle("CD") + annotation_scale(location = "bl", width_hint = 0.5) + annotation_scale(location = "bl", width_hint = 0.5)
plot(g)

## Inverse Power Relationship
The inverse power relationship as a typical functional approach to creating distance weights for a weights matrix W. 
When this relationship is used in spatial analysis and geostatistics it is often referred to as an inverse-distance weighting. To get a better conceptual and visual understanding of how an inverse power relationship works and what it looks like, you will create some data and graph it. 

Next we will create an object `d` to represent distance and define it as a vector of numbers from 1 to 50. In R you can create a sequence of numbers using a colon `:`. 

In [ ]:
d <- 1:50
d

Our inverse distance weighting is best represented with the following equation:

![image](equation1.png)

where the inverse distance simply means that we are going to divide `1` by our distance `d`. So if our distance was 10 then our inverse distance of 10 would be 1/10 or 0.1. The distance `d` that we will measure is going to be between point `i` and point `j`, which is why they are subscripted beside `d`. We also have the option of raising our distance by a power alpha `α`, which is why this is also called an `inverse power relationship`. When we raise a number to the power of `1`, it is equivalent to not having a power applied. The result of our inverse distance function is a weight <b>w<sub>ij</sub></b>, which is the weight applied to observation `i` with distance `d` from observation `j`.  

Next we will create three versions of this inverse power relationship for α=0.5, α = 1, and α=2 and assign them to variables labeled `wij_0_5`, `wij_1`, `wij_2`. Because `d` is a vector of fifty elements, the contents of w<sub>ij</sub> will contain fifty elements, which are the values for w<sub>ij</sub> at each distance. 

Lets also plot the inverse distance with `α = 1` in red.

In [ ]:
wij_0_5 <- 1/(d^0.5)
wij_1 <- 1/(d^1)
wij_2 <- 1/(d^2)

plot(d, wij_1, col="red")

Next, lets plot all three `α`s on the same graph using `ggplot2`. As you look at the figure take a minute to look at how the weight values change with increasing distance.

In [ ]:
df_idw <- data.frame(d, wij_0_5, wij_1, wij_2)

ggplot(data=df_idw, aes(x=d))+
  geom_line(aes(y=wij_0_5, colour="0.5")) +
  geom_line(aes(y=wij_1, colour="1")) +
  geom_line(aes(y=wij_2, colour="2")) +
  ylab(label="weights") +
  xlab("Distance") + 
  ggtitle("Inverse Distance Weighting Function") +
  theme(legend.position = c(.80,.80), legend.margin = margin (6,6,6,6)) + labs(colour = "alpha")

### Pairwise Two-Way Relationship - Weights Matrix

When creating a weights matrix for use in local statistics, geographically weighted regression, and other statistical metrics and models that require spatial structure, the matrix often has to be in a symmetrical form. Here you will perform a simple calculation in R to illustrate your understanding of one approach for creating a pairwise two-way relationship among locations by averaging two one-way relationships. 

Lets create a new object `W` and define it as the following:

In [ ]:
W <- data.frame(c(0,6,1),c(3,0,2),c(1,3,0))

Print out the contents of the data frame. You should notice that the column names are not very informative or appealing. 

In [ ]:
print(W)

We can create names and assign them to the columns using the `colnames()` function as follows:

In [ ]:
colnames(W) <- c("A","B","C")
print(W)

Next we will transpose the matrix `W` using the built-in transpose function `t()` to create a second non-symmetrical matrix for us to work with. We will call this second matrix `WT`.

In [ ]:
WT <- t(W)
print(WT)

To calculate the average of W and W<sup>T</sup>, we will use the following equation W<sub>final</sub>=(W+W<sup>T</sup>)/2. This will add each matching cell in the two data frames together and then divide all values within by 2. We will store our result in `WFinal`. We will also use the `rownames()` function to add the same labels to our rows.

In [ ]:
WFinal <- (W + WT)/2
rownames(WFinal) <- c("A", "B", "C")
print(WFinal)

As you can see the matrix is now symmetrical and (reading from row to column) the weight from `A` to `B` is now the same as the weight from `B` to `A`.



## Creating Areal Unit Count Data

In this workbook we wish to use spatial analyses and models that do not work well with point data. However, point data can be linked to areal data by aggregating the number of points per spatial unit (e.g., Census Division). One approach to doing this is to first overlay the point layer with the polygon layer and then summarize the points by the unique identifier of the polygons. The result of this overlay and summarize process will produce a count of the number of points in each polygon. We can then append our new count data into the polygon attribute table.

To achieve this task, we use the `over()` function to overlay the Home Depot stores point data `OHD` with the Census Division polygons `CD`. We assign the result of the overlay to `OHD_CD` and print that result.

In [ ]:
OHD_CD <- over(OHD, CD)
print(OHD_CD)

Then we can summarize the `OHD_CD` layer by the unique identifier (UID) representing each CD in the province (i.e., CDUID) using the following:

In [ ]:
HDStore_CD_tabulation <- table(OHD_CD$CDUID)
print (HDStore_CD_tabulation)

At this point our table only contains information about census units that have one or more stores within their boundary. Lets do some more confirmation here, using the `nrow()` function. Lets print out the total number of Census Units, the number of census units with one or more stores, and the number of census units without a store. We will use the `paste()` function to combine the text and outputs togehter.

In [ ]:
print(paste("Total number of census units =", nrow(CD), "."))
print(paste("Number of census units with stores =", nrow(HDStore_CD_tabulation), "."))
print(paste("Number of census units without stores =", nrow(CD) - nrow(HDStore_CD_tabulation), "."))
print(paste("Total number of stores =", nrow(OHD), "."))

Next, we will use the `data.frame()` function to convert our summary table to a data.frame so that we can add column names to our table using `colnames()`. Then we will use the `merge()` function to join our summary table data to our census polygon data based on the `CDUID` attribute. Where there is no match between the summary table and the polygon data, the `StoreCount` attribute received an `NA` value. So in the last step in this next code block, we find those `NA` values in the `StoreCount` attribute data and give them a value of zero. If you're interested, you can uncomment the `summary()` function to review some details about our new `mergedData` object.

In [ ]:
HDStore_CD_df <- data.frame(HDStore_CD_tabulation)
colnames(HDStore_CD_df) <- c("CDUID", "StoreCount")

mergedData <- merge(x=CD, y=HDStore_CD_df, by.x = c('CDUID'), by.y = c('CDUID'), all = TRUE)
mergedData[is.na(mergedData$StoreCount)] <- 0

#summary(mergedData)

There are a variety of different ways to plot data in R using a several different libraries. On my Macbook pro, with our census data, I've found that using the standard plot function `plot(mergedData)` seems to take about 5 minutes to plot. The `sp` library function `spplot()` works slightly faster with `spplot(mergedData, "StoreCount")`, which allows us to plot the `StoreCount` attribute. However, it seems that the `tmap` library is plotting our data the fastest (at least on my laptop). Be patient if you execute the next cell to plot the outcome of your point data aggregated by census unit. If you are plotting this on the Math JupyterHub then it should be much quicker. If you are remote desktopping into the ENV lab then you will not be able to execute the next code block as they do not have the `tmap` library installed.

In [ ]:
#install.packages("tmap")
library(tmap)
tm_shape(mergedData) + tm_polygons("StoreCount", style="cat", title="Store count \nby Census Unit")

## Hot Spot Analysis

Using our `mregedData`, we will conduct a hotspot analysis to create a surface that illustrates where there is higher and lower density/intensity of stores located. Before running the next few code blocks, hypothesize about where you think the hotspots are located. We will run a hotspot analysis using the `Local G` statistic. First we will need to load the `spdep` library to access the `poly2nb()`, `nb2listw()`, and `localG()` functions.

In [ ]:
#install.packages("spdep")
library(spdep)

The `poly2nb()` function builds a list of neighbours for each polygon. Here we will <b>use the Queens case, which specifies that any shared point between two polygons make them neighbours</b>.

In [ ]:
censusUnit_nb <- poly2nb(mergedData, queen = TRUE)

The `nb2listw()` function uses the neighbours list to generate spatial weights based on the connectivity between a polygon and its neighbours.

In [ ]:
censusUnit_nb_listw <- nb2listw(censusUnit_nb, zero.policy = TRUE)

Finally, we will use the `localG()` function from the `spdep` library to calculate the `Gi` statistic for each census unit polygon. The `localG()` function requires our data and `StoreCount` attribute, our list of weights `censusUnit_listw`, and the final parameter `zero.policy = TRUE` allows a polygon to have zero neighbours. 

In [ ]:
Gi <- localG(mergedData$StoreCount, censusUnit_nb_listw, zero.policy = FALSE)
Gcuts <- cut(Gi, 5)
Gcutsi <- as.integer(Gcuts)
cols <- rev(gray(seq(0,1,.2)))
plot(mergedData, col=cols[Gcutsi])
legend('bottomleft', levels(Gcuts), fill=cols, title = "Local Gi values")

Above, we used the local Gi statistic to identify if there was clustering in store counts among census units. A critical component to that analysis included how we define the neighbourhood or neighbours for each polygon. We used the Queen's case, which in the `spdep` defintion defines a neighbouring polygon as one that shares any point along the boarder of the host polygon. 

To see if the identified clusters are dependent on our definition of neighbourhood, lets repeat the process by <b>defining the neighbours of a polygon as the nearest k neighbours</b> and set k to 3. To do this we will use the `knearneigh()` function, which is also from the `spdep` library. Unfortunately this function will not work with polygon data. Therefore we use the `coordinates()` function to identify the polygon centroids and then the `knearneigh()` will identify the 1<sup>st</sup>, 2<sup>nd</sup>, 3<sup>rd</sup>, and 4<sup>th</sup> nearest census unit centroids. Then similar to using the `poly2nb()` function to generate a list of neighbours for each census unit, we will use the `knn2nb()` function to generate a list of our k nearest neighbours. Lastly, we use this list to generate our weights with the `nb2listw()` function.  

In [ ]:
censusUnit_knn <- knearneigh(coordinates(mergedData), k=3)
censusUnit_knnl <- knn2nb(censusUnit_knn)
censusUnit_knn_listw <- nb2listw(censusUnit_knnl, zero.policy = TRUE)

Now that we have a list of k nearest neighbour weights, lets <b>recompute the local Gi statistic</b> and see if the clustering of stores among census units changes with a change in the definition of neighbourhood. Remember, the plots can take some time.

In [ ]:
Gi <- localG(mergedData$StoreCount, censusUnit_knn_listw, zero.policy = FALSE)
Gcuts <- cut(Gi, 5)
Gcutsi <- as.integer(Gcuts)
cols <- rev(gray(seq(0,1,.2)))
plot(mergedData, col=cols[Gcutsi])
legend('bottomleft', levels(Gcuts), fill=cols, title = "Local Gi values")

As a final hotspot analysis using the local Gi statistic, we will <b>define our neighbourhood as comprising those neighbours within a specific Euclidean distance</b>. To do this we will use the same approach as above, except using the `dnearneigh()` function from the `spdep` package. While we can use any distance as our maximum distance `maxdist`, lets set the distance of our neighbour search to the average nearest neighbour distance. We can do this by using the `nndist()` function on the polygon centroids, acquired using the `coordinates()` function. In the `nndist()` function `k` again represents the number of neighbours we are interested in, which in this case is just one `k=1`. Then we can take the `mean()` of those nearest neighbour distances and assign it to our variable `maxSearchDist`. 

In [ ]:
maxSearchDist <- mean(nndist(coordinates(mergedData), k=1))

Now we can use the `dnearneigh()` function to obtain a list of neighbours for each census unit from `0` to our maximum search distance `maxSearchDist`. Then we can create weights for these distances using the `nb2listw()` function as we did above. 

In [ ]:
censusUnit_nbd <- dnearneigh(coordinates(mergedData), 0, maxSearchDist)
censusUnit_nbd_listw <- nb2listw(censusUnit_nbd, zero.policy = TRUE)

Now that we have a list of neighbours and their weights, we can recompute the local Gi statistic and see if the clustering of stores among census units changes when we <b>define our neighbourhood as those polygon centroids within our maximum search distance</b> `maxSearchDist`. Remember, the plots can take some time.

In [ ]:
Gi <- localG(mergedData$StoreCount, censusUnit_nbd_listw, zero.policy = FALSE)
Gcuts <- cut(Gi, 5)
Gcutsi <- as.integer(Gcuts)
cols <- rev(gray(seq(0,1,.2)))
plot(mergedData, col=cols[Gcutsi])
legend('bottomleft', levels(Gcuts), fill=cols, title = "Local Gi values")

# Congratulations!

**You have reached the end of the Inverse Distance and Local Statistics notebook**